# Docking

This notebook takes two files as input: a .pdb file for the receptor (aptamer) and a .sdf or .pdb file for the ligand. It is necessary to indicate the whole path for these files at the "Locate your files" step.

Both files will be used to reduce the molecules and obtain .pdbqt files, which will serve as input for AutoDock Vina to perform docking.

Before performing docking, it is necessary to create a parameters file, "param_file", containing the values of the parameters AutoDock Vina needs: xc, yc, zc, xs, ys, and zc. It is necessary to provide a unique name for this file for every experiment by writing it on the variable "param_file", and to provide the specific value for each parameter, which will value for the different molecules to dock.

As an output, this dock provides an .pdbqt file, which represents the results of the molecular docking simulation. They can be visualized using molecular visualization programs, such as Chimera or PyMOL.

### REMEMBER: change the Jupyter kernel to **Python (AptaLoop)**. See our README file for detailed information.

## 1.Import necessary packages

In [1]:
import importlib

package_name = "biopandas"

try:
    # Try to import the package
    importlib.import_module(package_name)
except ImportError:
    # If the package is not installed, install it using pip
    import subprocess
    subprocess.call(["pip", "install", package_name])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 3.4 MB/s eta 0:00:0000:0100:01


In [2]:
from biopandas.pdb import PandasPdb
import os

## 2. Create necessary directories


The original files will be moved to the directory "Original files", in order to not lose them. However, if they have been generated by MAWS or 3dRNA, these original files will need to be processed before AutoDock Vina can take them as input. The fixed aptamers' PDBs will be stored in "PDB_files"  The PDBQT files generated will be stored on "PDBQT_files".

For each Vina run, a paramaters' file will be generated and stored on "Vina_params", while the "Vina results" will be stored on "Vina_results".

In [3]:
%%bash

mkdir -p PDB_files
mkdir -p PDBQT_files
mkdir -p Original_files
mkdir -p Vina_results
mkdir -p Vina_params

## 3. Locate your files (pdb or sdf) in the Original_files directory

Indicate the path for the orignial Aptamer and Ligand files.

In [4]:
Aptamer_path="../../data/3dRNA_DNA_PFOA2.pdb"
Ligand_path="../../data/PFOA.sdf"

Function to extract the file name (without the extension)

In [5]:
def extract_file_name(input_file):
    # Get the base name of the file without the extension
    file_name = os.path.splitext(input_file)[0]
    return file_name

Aptamer's file and name.

In [6]:
Aptamer_file = os.path.basename(Aptamer_path) # 3dRNA_DNA_PFOA2.pdb
Aptamer_name = extract_file_name(Aptamer_file) # 3dRNA_DNA_PFOA2

Ligand's file and name

In [7]:
Ligand_file = os.path.basename(Ligand_path) # PFOA.pdb or PFOA.sdf

# Check if Aptamer_name ends with ".sdf"
if Ligand_path.endswith(".sdf"):
    format_ligand = "sdf"
# Check if Aptamer_name ends with ".pdb"
elif Ligand_file.endswith(".pdb"):
    format_ligand = "pdb"
# If neither ".sdf" nor ".pdb" is found at the end, you can provide a default value
else:
    print("Please, ensure that your ligand is in either .sdf or .pdb format")
    
# Check the format_ligand and extract Ligand_name accordingly
Ligand_name = extract_file_name(Ligand_file) # PFOA

In [8]:
%%bash -s "$Aptamer_path" "$Ligand_path"

cp $1 Original_files/
cp $2 Original_files/

## 4. Edit PDB files so that they are suitable for AutoDock Vina (Eliminate "5" and "3" in the first and last nucleotides).

Edit the original file

In [9]:
# Load the PDB file into a dataframe

ppdb = PandasPdb().read_pdb(Aptamer_path)
pdb_df = ppdb.df['ATOM']

# Edit the rows corresponding to the first residue
pdb_df.loc[pdb_df['residue_number'] == 1, 'residue_name'] = pdb_df.loc[pdb_df['residue_number'] == 1, 'residue_name'].str.extract('([A-Za-z]+)', expand=False)

# Edit the rows corresponding to the last residue
max_residue = pdb_df['residue_number'].max()
pdb_df.loc[pdb_df['residue_number'] == max_residue, 'residue_name'] = pdb_df.loc[pdb_df['residue_number'] == max_residue, 'residue_name'].str.extract('([A-Za-z]+)', expand=False)


Save the fixed PDB in the "PDB_files" directory

In [10]:
fixed_output_path = 'PDB_files/' + Aptamer_name + '_fixed.pdb'

In [11]:
# Create a PandasPdb object
ppdb_out = PandasPdb()

# Assign the modified DataFrame to the 'ATOM' attribute
ppdb_out.df['ATOM'] = pdb_df

# Save the modified DataFrame to a PDB file
output_pdb_path = fixed_output_path
ppdb_out.to_pdb(output_pdb_path, records=['ATOM'])

## 3.Make sure both receptor and ligand are reduced, and create PDBQT files

In [12]:
%%bash -s "$Aptamer_name" "$Ligand_name" "$format_ligand" "$fixed_output_path"

# Reduce the aptamer
obabel -i pdb $4 -o pdb -h > PDB_files/$1_H.pdb
# Get the pdbqt for the aptamer
obabel PDB_files/$1_H.pdb -O PDBQT_files/$1.pdbqt -p 7.4 -xr


# Reduce the ligand
# Check if $3 equals "sdf"
if [ "$3" = "sdf" ]; then
    obabel -i sdf Original_files/$2.sdf -o sdf -h > Original_files/$2_H.sdf
    obabel -isdf Original_files/$2_H.sdf -opdbqt -O PDBQT_files/$2.pdbqt
# Check if $3 equals "pdb"
elif [ "$3" = "pdb" ]; then
    obabel -i pdb Original_files/$2.pdb -o pdb -h > Original_files/$2_H.pdb
    obabel Original_files/$2_H.pdb -O PDBQT_files/$2.pdbqt -p 7.4

else
    # Handle other cases or provide an error message
    echo "Invalid value for \$3: $3"
    exit 1
fi

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


## 4.Run AutodockVina

In [13]:
%%bash -s "$Aptamer_name" "$Ligand_name"

# Create parameters' file

param_file=$1_params.txt
x_c=8.5605
y_c=88.4661
z_c=154.273
x_s=105.973
y_s=63.3577
z_s=67.2692
  
echo $x_c>Vina_params/$param_file
echo $y_c>>Vina_params/$param_file
echo $z_c>>Vina_params/$param_file
echo $x_s>>Vina_params/$param_file
echo $y_s>>Vina_params/$param_file
echo $z_s>>Vina_params/$param_file


# Initialize the variables

xc=""
yc=""
zc=""
xs=""
ys=""
zs=""

# Read the content of the file into variables using a loop
while IFS= read -r line; do
  if [ -z "$xc" ]; then
    xc="$line"
  elif [ -z "$yc" ]; then
    yc="$line"
  elif [ -z "$zc" ]; then
    zc="$line"
  elif [ -z "$xs" ]; then
    xs="$line"
  elif [ -z "$ys" ]; then
    ys="$line"
  elif [ -z "$zs" ]; then
    zs="$line"
  fi
done < Vina_params/"$param_file"

vina --receptor PDBQT_files/$1.pdbqt --ligand PDBQT_files/$2.pdbqt --center_x $xc --center_y $yc --center_z $zc --size_x $xs --size_y $ys --size_z $zs --out Vina_results/$1.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

Computing Vina grid ... done.
Performing docking (random seed: 32504079) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1    -5.86e-06          0          0
   2    -5.86e-06      51.08      54.23
   3    -5.86e-06      69.39      71.59
   4    -2.795e-06      27.07      28.64
   5            0      43.61      44.94
   6    0.0002269      26.97      28.42
   7    0.0003219      13.39      17.06
   8     0.000618      69.28      71.29
   9     0.003156      45.24      46.95
